# PyODB Performance Tests


In [1]:
import cProfile
from datetime import datetime
from pathlib import Path
from tqdm import tqdm

from src.pyodb.pyodb import PyODB
from test.test_models.complex_models import ComplexBasic, ComplexContainer, ComplexMulti
from test.test_models.primitive_models import PrimitiveBasic, PrimitiveContainer

In [3]:
def test_insert_performance():
    pyodb = PyODB(2)
    pyodb.add_type(ComplexBasic)
    pyodb.save(ComplexContainer())
    pyodb.add_type(ComplexMulti)

    for _ in tqdm(range(10)):
        pyodb.save_multiple([ComplexBasic() for _ in range(100)])
        pyodb.save_multiple([ComplexContainer() for _ in range(10)])
        pyodb.save_multiple([ComplexMulti() for _ in range(100)])

        pyodb.delete(ComplexBasic).gt(random_number = 0).commit()
        pyodb.remove_type(ComplexContainer)
        pyodb.add_type(ComplexContainer)
    del pyodb


base_path = Path(".profile/")
base_path.mkdir(755, exist_ok=True)

filepath = base_path / f"profile_insert_{datetime.now().strftime('%y.%m.%d-%H.%M.%S')}.prof"
cProfile.run("test_insert_performance()", filepath.as_posix())

100%|██████████| 10/10 [00:10<00:00,  1.03s/it]


In [2]:
def insert_base_data():
    pyodb = PyODB(persistent=True)
    pyodb.add_type(ComplexBasic)
    pyodb.add_type(ComplexContainer)
    pyodb.add_type(ComplexMulti)

    for _ in tqdm(range(10), desc="Inserting base data"):
        pyodb.save_multiple([ComplexBasic() for _ in range(100)])
        pyodb.save_multiple([ComplexContainer() for _ in range(10)])
        pyodb.save_multiple([ComplexMulti() for _ in range(100)])


def test_select_performance():
    pyodb = PyODB()
    for _ in tqdm(range(100), desc="Testing PrimitiveBasic"):
        pyodb.select(PrimitiveBasic).all()
    for _ in tqdm(range(100), desc="Testing PrimitiveContainer"):
        pyodb.select(PrimitiveContainer).all()
    for _ in tqdm(range(100), desc="Testing ComplexBasic"):
        pyodb.select(ComplexBasic).all()
    for _ in tqdm(range(100), desc="Testing ComplexContainer"):
        pyodb.select(ComplexContainer).all()


base_path = Path(".profile/")
base_path.mkdir(755, exist_ok=True)

filepath = base_path / f"profile_select_{datetime.now().strftime('%y.%m.%d-%H.%M.%S')}.prof"
insert_base_data()
cProfile.run("test_select_performance()", filepath.as_posix())

Testing ComplexContainer: 100%|██████████| 100/100 [00:00<00:00, 108.57it/s]
